In [ ]:
import csv
import math
import os
import warnings

from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
from Bio.PDB import (
    PDBParser,
    PPBuilder
)
from Bio.PDB.Polypeptide import (
    three_to_one,
    one_to_three
)

import pandas as pd
import numpy as np

from tqdm import tqdm

from common import (
    R,
    skempi_pdbs_dir,
    skempi_csv_path,
    pdb_ext,
    delta_g_path,
    id_seq_path
)

In [ ]:
df = pd.read_csv(skempi_csv_path, sep = ';')
df = df.dropna(subset = ['Affinity_mut_parsed', 'Affinity_wt_parsed', 'Temperature'])
df.columns

In [ ]:
# process temperature, like 298(assumed)
df["Temperature"] = df["Temperature"].str.slice(0,3)

In [ ]:
delta_g_data = []
id_seq_dict = {}

In [ ]:
for idx, row in tqdm(df.iterrows()):
    delta_g_item = {}
    
    # ========== parse skempi ==========
    pdb_entry = row["#Pdb"]
    temp = row["Temperature"]
    ent_name = pdb_entry[:4]
    model_chain_ids = pdb_entry.split('_')[1:]
    mutations_pdb = row["Mutation(s)_cleaned"].split(',')
    # just keep single point mutation records
    if len(mutations_pdb) != 1:
        continue
    mutation_pdb = mutations_pdb[0]
    delta_g_wt = R * float(temp) * math.log(float(row["Affinity_wt_parsed"]))
    delta_g_mut = R * float(temp) * math.log(float(row["Affinity_mut_parsed"]))
    delta_g_item["delta_g_wt"] = delta_g_wt
    delta_g_item["delta_g_mut"] = delta_g_mut
    delta_g_item["delta_delta_g"] = delta_g_mut - delta_g_wt
    # print(ent_name, model_chain_ids, mutation_pdb, delta_g_wt, delta_g_mut, temp)
    mut_chain_id, mut_residue_id, r_wt, r_mut = mutation_pdb[1], int(mutation_pdb[2:-1]), mutation_pdb[0], mutation_pdb[-1]
    # print(mut_chain_id, mut_residue_id, r_wt, r_mut)
    
    # ========== parse pdb ==========
    parser = PDBParser()
    structure = parser.get_structure(ent_name, os.path.join(skempi_pdbs_dir, "{}.{}".format(ent_name, pdb_ext)))
    # should be only one structure
    if len(structure) != 1:
        continue
    model = structure[0]
    
    # ========== build id -> seq map and pair -> delta g map ==========
    # wt pair and mut pair
    for pair in ["wt", "mut"]:
        for model_idx, model_chain_id in enumerate(model_chain_ids):
            model_name = "{}_{}".format(ent_name, model_chain_id)            
            model_seqs = ""
            # multiple chains in one protein
            for chain_id in model_chain_id:
                chain_entry = model[chain_id]
                # mutate
                if pair == "mut" and chain_entry.id == mut_chain_id:
                    residue = chain_entry[mut_residue_id]
                    pdb_wt = three_to_one(residue.resname)
                    if pdb_wt != r_wt:
                        print("WARN: PDB wt residue {} not equals to skempi wt residue {}".format(pdb_wt, r_wt))
                    residue.resname = one_to_three(r_mut)
                    model_name = "{}_{}{}{}{}".format(model_name, r_wt, mut_chain_id, mut_residue_id, r_mut)
                builder = PPBuilder()
                for pp in builder.build_peptides(chain_entry):
                    model_seqs += pp.get_sequence()
            delta_g_item["{}_name_{}".format(pair, model_idx)] = model_name
            # id -> seq map
            if model_name not in id_seq_dict:
                id_seq_dict[model_name] = str(model_seqs)
    
    delta_g_data.append(delta_g_item)

In [ ]:
df_delta_g = pd.DataFrame(delta_g_data)
df_delta_g

In [ ]:
id_seq_data = [
    {"id": k, "seqs": v}
    for k, v in id_seq_dict.items()
]
df_id_seq = pd.DataFrame(id_seq_data)
df_id_seq

In [ ]:
df_delta_g.to_csv(delta_g_path, index=0)
df_id_seq.to_csv(id_seq_path, index=0)